In [1]:
import neptune.new as neptune

In [2]:
project_name = 'PESC-ATIS/FairnessSeminary-21-07-21'

project = neptune.get_project(
    name=project_name,
    api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1ZGUxY2IwMy1kOTMzLTRjMTUtYjAxYy01MWE2MmMyYzQ0ZmYifQ==')

In [3]:
run_table_df = project.fetch_runs_table().to_pandas()

run_table_df.head()

,sys/creation_time,sys/description,sys/failed,sys/hostname,sys/id,sys/modification_time,sys/monitoring_time,sys/name,sys/owner,sys/ping_time,...,parameters/hyperparameters/hidden_size_2,parameters/hyperparameters/num_layers,parameters/num-classes,parameters/num-features,source_code/entrypoint,source_code/git,source_code/integrations/neptune-tensorflow-keras,monitoring/cpu,monitoring/memory,monitoring/stdout
0,2021-07-20 13:40:38.087000+00:00,,False,meriadoc,FAIR-168,2021-07-20 13:41:03.258000+00:00,624,arbitrary_correction_batch,ygorcanalli,2021-07-20 13:41:03.258000+00:00,...,32.0,3.0,2.0,108.0,pydevconsole.py,5f179c1218825edfc67c357ff20e263d8bed6163,0+untagged.67.g5f179c1.dirty,26.6,9.001030,"Shutting down background jobs, please wait a m..."
1,2021-07-20 13:40:37.209000+00:00,,False,meriadoc,FAIR-167,2021-07-20 13:41:02.733000+00:00,624,arbitrary_correction_batch,ygorcanalli,2021-07-20 13:41:02.733000+00:00,...,32.0,3.0,2.0,108.0,pydevconsole.py,5f179c1218825edfc67c357ff20e263d8bed6163,0+untagged.67.g5f179c1.dirty,27.7,8.974144,"Shutting down background jobs, please wait a m..."
2,2021-07-20 13:40:06.857000+00:00,,False,meriadoc,FAIR-166,2021-07-20 13:40:35.381000+00:00,627,arbitrary_correction_batch,ygorcanalli,2021-07-20 13:40:35.381000+00:00,...,32.0,3.0,2.0,108.0,pydevconsole.py,5f179c1218825edfc67c357ff20e263d8bed6163,0+untagged.67.g5f179c1.dirty,16.9,8.813175,"Shutting down background jobs, please wait a m..."
3,2021-07-20 13:40:06.097000+00:00,,False,meriadoc,FAIR-165,2021-07-20 13:40:38.959000+00:00,632,arbitrary_correction_batch,ygorcanalli,2021-07-20 13:40:38.959000+00:00,...,32.0,3.0,2.0,108.0,pydevconsole.py,5f179c1218825edfc67c357ff20e263d8bed6163,0+untagged.67.g5f179c1.dirty,15.7,8.718899,"Shutting down background jobs, please wait a m..."
4,2021-07-20 13:40:05.848000+00:00,,False,meriadoc,FAIR-164,2021-07-20 13:40:37.330000+00:00,630,arbitrary_correction_batch,ygorcanalli,2021-07-20 13:40:37.330000+00:00,...,32.0,3.0,2.0,108.0,pydevconsole.py,5f179c1218825edfc67c357ff20e263d8bed6163,0+untagged.67.g5f179c1.dirty,15.8,8.717907,"Shutting down background jobs, please wait a m..."


In [4]:
run_table_df.rename(columns={'metrics/eval_accuracy': 'accuracy',
                     'model/params/dataset': 'dataset',
                     'model/params/robust-method': 'robust-method',
                     'metrics/eval_accuracy': 'accuracy',
                     'metrics/discrimination': 'discrimination',
                     'metrics/epoch/val_MatthewsCorrelationCoefficient': 'mcc',
                     'model/params/loss-args' : 'transition'
                    }, inplace=True)


In [10]:
run_table_df['acc/disc'] = run_table_df['accuracy']/run_table_df['discrimination']
run_table_df['mcc/disc'] = run_table_df['mcc']/run_table_df['discrimination']


group_criteria = ['dataset', 'robust-method', 'transition']
agg_criteria = ({'accuracy': ['mean', 'std'],
                 'discrimination': ['mean', 'std'],
                 'mcc': ['mean', 'std'],
                 'acc/disc': ['mean', 'std']})

results = run_table_df.groupby(group_criteria)\
                        .agg(agg_criteria)\
                        .sort_values(by=('accuracy', 'mean'), ascending=False)
results

accuracy  \
                                                                              mean   
dataset robust-method transition                                                     
income  None          None                                                0.777532   
        fair_forward  [[array([[0.8 , 0.2 ],\n       [0.02, 0.98]]), ...  0.566243   
                      [[array([[0.85 , 0.15 ],\n       [0.015, 0.985]...  0.561984   
                      [[array([[0.8 , 0.2 ],\n       [0.02, 0.98]]), ...  0.552239   
                      [[array([[0.8 , 0.2 ],\n       [0.02, 0.98]]), ...  0.549598   
                      [[array([[0.8 , 0.2 ],\n       [0.02, 0.98]]), ...  0.544008   
                      [[array([[0.85 , 0.15 ],\n       [0.015, 0.985]...  0.542903   
                      [[array([[0.9 , 0.1 ],\n       [0.01, 0.99]]), ...  0.542452   
                      [[array([[0.85 , 0.15 ],\n       [0.015, 0.985]...  0.542248   
                      [[array([[0.8 , 0.2 ],\n       [0.02, 0.98]]), ...  0.541674   
                      [[array([[0.85 , 0.15 ],\n       [0.015, 0.985]...  0.539074   
                      [[array([[0.9 , 0.1 ],\n       [0.01, 0.99]]), ...  0.538562   
                      [[array([[0.9 , 0.1 ],\n       [0.01, 0.99]]), ...  0.537958   
                      [[array([[0.85 , 0.15 ],\n       [0.015, 0.985]...  0.535737   
                      [[array([[0.9 , 0.1 ],\n       [0.01, 0.99]]), ...  0.533608   
                      [[array([[0.95 , 0.05 ],\n       [0.005, 0.995]...  0.531755   
                      [[array([[0.95 , 0.05 ],\n       [0.005, 0.995]...  0.530189   
                      [[array([[0.95 , 0.05 ],\n       [0.005, 0.995]...  0.524313   
                      [[array([[0.9 , 0.1 ],\n       [0.01, 0.99]]), ...  0.523739   
                      [[array([[0.95 , 0.05 ],\n       [0.005, 0.995]...  0.523248   
                      [[array([[1., 0.],\n       [0., 1.]]), array([[...  0.516707   
                      [[array([[1., 0.],\n       [0., 1.]]), array([[...  0.516338   
                      [[array([[1., 0.],\n       [0., 1.]]), array([[...  0.513252   
                      [[array([[0.95 , 0.05 ],\n       [0.005, 0.995]...  0.509582   
                      [[array([[1., 0.],\n       [0., 1.]]), array([[...  0.508691   
                      [[array([[1., 0.],\n       [0., 1.]]), array([[...  0.497420   

                                                                                    \
                                                                               std   
dataset robust-method transition                                                     
income  None          None                                                0.008887   
        fair_forward  [[array([[0.8 , 0.2 ],\n       [0.02, 0.98]]), ...  0.013950   
                      [[array([[0.85 , 0.15 ],\n       [0.015, 0.985]...  0.004114   
                      [[array([[0.8 , 0.2 ],\n       [0.02, 0.98]]), ...  0.011545   
                      [[array([[0.8 , 0.2 ],\n       [0.02, 0.98]]), ...  0.002021   
                      [[array([[0.8 , 0.2 ],\n       [0.02, 0.98]]), ...  0.008808   
                      [[array([[0.85 , 0.15 ],\n       [0.015, 0.985]...  0.025972   
                      [[array([[0.9 , 0.1 ],\n       [0.01, 0.99]]), ...  0.015722   
                      [[array([[0.85 , 0.15 ],\n       [0.015, 0.985]...  0.012513   
                      [[array([[0.8 , 0.2 ],\n       [0.02, 0.98]]), ...  0.012128   
                      [[array([[0.85 , 0.15 ],\n       [0.015, 0.985]...  0.017993   
                      [[array([[0.9 , 0.1 ],\n       [0.01, 0.99]]), ...  0.028157   
                      [[array([[0.9 , 0.1 ],\n       [0.01, 0.99]]), ...  0.022888   
                      [[array([[0.85 , 0.15 ],\n       [0.015, 0.985]...  0.017643   
                      [[array([[0.9 , 0.1 ],\n       [0.01, 0.99]]), ...  0.022093   
                      [[array([